In [8]:
import pandas as pd
df = pd.read_excel('/home/esp/data_analyze/05.02.2025/data/init/Metadata_BPD.xlsx')
df.to_csv('/home/esp/data_analyze/05.02.2025/data/init/Metadata_BPD.csv', index=False, sep=',')


In [6]:
import pandas as pd
import numpy as np
import lib
import json

DATA_PATH = '../data/init/'
NAME = 'rats_1'
PATH_OUTPUT_DATA = f'../data/output/{NAME}'
META = 'Metadata_BPD'
lib.GROUP = 'GROUP'

otu_df = lib.read_data(f'{DATA_PATH}OTU.txt', sep='\t')
hiera_blast = lib.read_data(f'{DATA_PATH}hiera_BLAST.txt', sep='\t')
meta_map_init = lib.read_data(f'{DATA_PATH}in.map', sep='\t')
meta = lib.read_data(f'{DATA_PATH}{META}.csv',  sep=',')
genus_df = lib.read_data(f'{DATA_PATH}Genus.txt', sep='\t')
phylum_df = lib.read_data(f'{DATA_PATH}Phylum.txt')

with open(f'{DATA_PATH}OTU.biom', 'r') as f:
    BIOM_DATA = json.load(f)

In [7]:
meta_group = meta['GROUP'].unique().tolist()
pattern = r'^k__\?\|p__\?\|c__\?\|o__\?\|f__\?\|g__\?\|s__$'
FILTERED_GROUPS = meta_group

meta_map_df = lib.preprocess_meta_map(meta_map_init)
meta_map_df = meta_map_df[~meta_map_df['#SampleID'].isin(['29D2', '32D2', '24A2'])]

# Переименовываем столбец в meta, чтобы ключ слияния был 'ID'
meta = meta.rename(columns={'#SampleID': 'ID'})

# Убираем префикс "S" из значений столбца '#SampleID' в meta_map_df, чтобы они совпадали с otu_df
meta_map_df['ID'] = meta_map_df['ID'].str.replace('^S', '', regex=True)

merged_meta = pd.merge(meta_map_df, meta, on='ID')
print("Слияние meta_map и meta:")
print(merged_meta.head())

prepare_to_maaslin2, merged_otu_df = lib.merge_and_prepare_datasets(meta_map_df, meta, otu_df, None)

merged_otu_df = lib.update_asv_columns(merged_otu_df, BIOM_DATA)

long_format_df = lib.melt_and_prepare_taxonomy(merged_otu_df)
taxon_abundance = lib.rel_abundance(long_format_df)
safe_df = taxon_abundance[taxon_abundance['GROUP'].isin(FILTERED_GROUPS)]
safe_df = safe_df[~safe_df['Taxonomy'].str.match(pattern)]


# filtered_df = lib.filter_threshold(taxon_abundance)
# filtered_df_5 = lib.filter_threshold(taxon_abundance, 0.01)
# filtered_remove_df = lib.remove_low_variance_taxa(filtered_df, 0.00001)
# safe_data = filtered_remove_df[filtered_remove_df['GROUP'].isin(FILTERED_GROUPS)]
#to_maaslin_date = lib.to_maaslin_data(otu_df, prepare_to_maaslin2)
prepare_to_maaslin2 = prepare_to_maaslin2[prepare_to_maaslin2['GROUP'].isin(FILTERED_GROUPS)]



lib.SAFE=True
if lib.SAFE:
    prepare_to_maaslin2.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/{NAME}.tsv', sep='\t')
    #to_maaslin_date.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/maaslin_data.tsv', sep='\t')
    safe_df.to_csv(f'{PATH_OUTPUT_DATA}/out_data/{NAME}_out.csv', sep='\t')
    #for_lefse_filtered = lib.final_lefse_df(filtered_remove_df, PATH_OUTPUT_DATA, "ALL", lib.SAFE)

Слияние meta_map и meta:
  #SampleID   ID fastqFile       GROUP Unnamed: 3
0       10A  10A     MAM-J  OB-BPD/DS    резекция
1       10B  10B     MAM-I  OB-BPD/DS    резекция
2       10C  10C     MAMLI  OB-BPD/DS    резекция
3       10D  10D     MAM-D  OB-BPD/DS    резекция
4       10F  10F     Fecal  OB-BPD/DS    резекция


In [8]:
safe_df.head()

,#SampleID,Taxonomy,GROUP,Value,Total,RelativeAbundance
1,10A,k__Archaea|p__Crenarchaeota|c__Bathyarchaeia|o...,OB-BPD/DS,0,92085,0.0
2,10A,k__Archaea|p__Crenarchaeota|c__Bathyarchaeia|o...,OB-BPD/DS,0,92085,0.0
3,10A,k__Archaea|p__Crenarchaeota|c__Nitrososphaeria...,OB-BPD/DS,0,92085,0.0
4,10A,k__Archaea|p__Euryarchaeota|c__Methanobacteria...,OB-BPD/DS,0,92085,0.0
5,10A,k__Archaea|p__Euryarchaeota|c__Methanobacteria...,OB-BPD/DS,0,92085,0.0


### GENUS


In [9]:
import pandas as pd
import numpy as np
import lib
import json

DATA_PATH = '../data/init/'
NAME = 'genus'
PATH_OUTPUT_DATA = f'../data/output/{NAME}'
META = 'Metadata_BPD'
lib.GROUP = 'GROUP'

otu_df = lib.read_data(f'{DATA_PATH}OTU.txt', sep='\t')
hiera_blast = lib.read_data(f'{DATA_PATH}hiera_BLAST.txt', sep='\t')
meta_map_init = lib.read_data(f'{DATA_PATH}in.map', sep='\t')
meta = lib.read_data(f'{DATA_PATH}{META}.csv',  sep=',')
genus_df = lib.read_data(f'{DATA_PATH}Genus.txt', sep='\t')
phylum_df = lib.read_data(f'{DATA_PATH}Phylum.txt')

with open(f'{DATA_PATH}OTU.biom', 'r') as f:
    BIOM_DATA = json.load(f)

In [10]:
meta_group = meta['GROUP'].unique().tolist()
pattern = r'^k__\?\|p__\?\|c__\?\|o__\?\|f__\?\|g__\?\|s__$'
FILTERED_GROUPS = meta_group

meta_map_df = lib.preprocess_meta_map(meta_map_init)
meta_map_df = meta_map_df[~meta_map_df['#SampleID'].isin(['29D2', '32D2', '24A2'])]

# Переименовываем столбец в meta, чтобы ключ слияния был 'ID'
meta = meta.rename(columns={'#SampleID': 'ID'})

# Убираем префикс "S" из значений столбца '#SampleID' в meta_map_df, чтобы они совпадали с genus_df
meta_map_df['ID'] = meta_map_df['ID'].str.replace('^S', '', regex=True)

merged_meta = pd.merge(meta_map_df, meta, on='ID')
print("Слияние meta_map и meta:")
print(merged_meta.head())

prepare_to_maaslin2, merged_genus_df = lib.merge_and_prepare_datasets(meta_map_df, meta, genus_df, None)

merged_genus_df = lib.update_asv_columns(merged_genus_df, BIOM_DATA)

long_format_df = lib.melt_and_prepare_taxonomy(merged_genus_df)
taxon_abundance = lib.rel_abundance(long_format_df)
safe_df = taxon_abundance[taxon_abundance['GROUP'].isin(FILTERED_GROUPS)]
safe_df = safe_df[~safe_df['Taxonomy'].str.match(pattern)]


# filtered_df = lib.filter_threshold(taxon_abundance)
# filtered_df_5 = lib.filter_threshold(taxon_abundance, 0.01)
# filtered_remove_df = lib.remove_low_variance_taxa(filtered_df, 0.00001)
# safe_data = filtered_remove_df[filtered_remove_df['GROUP'].isin(FILTERED_GROUPS)]
#to_maaslin_date = lib.to_maaslin_data(genus_df, prepare_to_maaslin2)
prepare_to_maaslin2 = prepare_to_maaslin2[prepare_to_maaslin2['GROUP'].isin(FILTERED_GROUPS)]



lib.SAFE=True
if lib.SAFE:
    prepare_to_maaslin2.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/{NAME}.tsv', sep='\t')
    #to_maaslin_date.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/maaslin_data.tsv', sep='\t')
    safe_df.to_csv(f'{PATH_OUTPUT_DATA}/out_data/{NAME}_out.csv', sep='\t')
    #for_lefse_filtered = lib.final_lefse_df(filtered_remove_df, PATH_OUTPUT_DATA, "ALL", lib.SAFE)

Слияние meta_map и meta:
  #SampleID   ID fastqFile       GROUP Unnamed: 3
0       10A  10A     MAM-J  OB-BPD/DS    резекция
1       10B  10B     MAM-I  OB-BPD/DS    резекция
2       10C  10C     MAMLI  OB-BPD/DS    резекция
3       10D  10D     MAM-D  OB-BPD/DS    резекция
4       10F  10F     Fecal  OB-BPD/DS    резекция


In [11]:
safe_df

,#SampleID,Taxonomy,GROUP,Value,Total,RelativeAbundance
0,10A,Archaea|p_Crenarchaeota|c_Bathyarchaeia|o_uncu...,OB-BPD/DS,0,92085,0.000000
1,10A,Archaea|p_Crenarchaeota|c_Bathyarchaeia|o_uncu...,OB-BPD/DS,0,92085,0.000000
2,10A,Archaea|p_Crenarchaeota|c_Nitrososphaeria|o_Ni...,OB-BPD/DS,0,92085,0.000000
3,10A,Archaea|p_Euryarchaeota|c_Methanobacteria|o_Me...,OB-BPD/DS,0,92085,0.000000
4,10A,Archaea|p_Euryarchaeota|c_Methanobacteria|o_Me...,OB-BPD/DS,0,92085,0.000000
...,...,...,...,...,...,...
109986,7D,Bacteria|p_Verrucomicrobiota|c_Verrucomicrobia...,OB-BPD/DS,0,168130,0.000000
109987,7D,Bacteria|p_Verrucomicrobiota|c_Verrucomicrobia...,OB-BPD/DS,0,168130,0.000000
109988,7D,Bacteria|p_WPS-2,OB-BPD/DS,0,168130,0.000000
109989,7D,Eukaryota,OB-BPD/DS,0,168130,0.000000


### Genus Subgroup

In [2]:
import pandas as pd
df = pd.read_excel('/home/esp/data_analyze/05.02.2025/data/init/New_Metadata_BPD.xlsx')
df.to_csv('/home/esp/data_analyze/05.02.2025/data/init/New_Metadata_BPD.csv', index=False, sep=',')


In [1]:
import pandas as pd
import numpy as np
import lib
import json

DATA_PATH = '../data/init/'
NAME = 'genus_subgroup'
PATH_OUTPUT_DATA = f'../data/output/{NAME}'
META = 'New_Metadata_BPD'
lib.GROUP = 'Group'

otu_df = lib.read_data(f'{DATA_PATH}OTU.txt', sep='\t')
hiera_blast = lib.read_data(f'{DATA_PATH}hiera_BLAST.txt', sep='\t')
meta_map_init = lib.read_data(f'{DATA_PATH}in.map', sep='\t')
meta = lib.read_data(f'{DATA_PATH}{META}.csv',  sep=',')
genus_df = lib.read_data(f'{DATA_PATH}Genus.txt', sep='\t')
phylum_df = lib.read_data(f'{DATA_PATH}Phylum.txt')

with open(f'{DATA_PATH}OTU.biom', 'r') as f:
    BIOM_DATA = json.load(f)

In [9]:
meta_group = meta['Group'].unique().tolist()
pattern = r'^k__\?\|p__\?\|c__\?\|o__\?\|f__\?\|g__\?\|s__$'
FILTERED_GROUPS = meta_group

meta_map_df = lib.preprocess_meta_map(meta_map_init)
meta_map_df = meta_map_df[~meta_map_df['#SampleID'].isin(['29D2', '32D2', '24A2'])]

# Переименовываем столбец в meta, чтобы ключ слияния был 'ID'
meta = meta.rename(columns={'#SampleID': 'ID'})

# Убираем префикс "S" из значений столбца '#SampleID' в meta_map_df, чтобы они совпадали с genus_df
meta_map_df['ID'] = meta_map_df['ID'].str.replace('^S', '', regex=True)

merged_meta = pd.merge(meta_map_df, meta, on='ID')
print("Слияние meta_map и meta:")
print(merged_meta.head())

prepare_to_maaslin2, merged_genus_df = lib.merge_and_prepare_datasets(meta_map_df, meta, genus_df, None)

merged_genus_df = lib.update_asv_columns(merged_genus_df, BIOM_DATA)

long_format_df = lib.melt_and_prepare_taxonomy(merged_genus_df)
taxon_abundance = lib.rel_abundance(long_format_df)
safe_df = taxon_abundance[taxon_abundance['GROUP'].isin(FILTERED_GROUPS)]
safe_df = safe_df[~safe_df['Taxonomy'].str.match(pattern)]


# filtered_df = lib.filter_threshold(taxon_abundance)
# filtered_df_5 = lib.filter_threshold(taxon_abundance, 0.01)
# filtered_remove_df = lib.remove_low_variance_taxa(filtered_df, 0.00001)
# safe_data = filtered_remove_df[filtered_remove_df['GROUP'].isin(FILTERED_GROUPS)]
#to_maaslin_date = lib.to_maaslin_data(genus_df, prepare_to_maaslin2)
prepare_to_maaslin2 = prepare_to_maaslin2[prepare_to_maaslin2['GROUP'].isin(FILTERED_GROUPS)]

safe_df_with_subgroup = lib.attach_subgroup_column(
    safe_df=safe_df,
    meta_df=meta,            # ваш DataFrame из CSV с колонкой "Subgroup"
    meta_id='ID',            # как называется ID в meta (по дефолту 'ID')
    df_id='#SampleID',       # как называется ID в safe_df
    subgroup_col='Subgroup'  # какая колонка из meta хотим добавить
)
safe_df_with_subgroup = safe_df_with_subgroup[~safe_df_with_subgroup['Taxonomy'].str.startswith('noHit|p_')]


lib.SAFE=True
if lib.SAFE:
    prepare_to_maaslin2.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/{NAME}.tsv', sep='\t')
    #to_maaslin_date.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/maaslin_data.tsv', sep='\t')
    safe_df_with_subgroup.to_csv(f'{PATH_OUTPUT_DATA}/out_data/{NAME}_out.csv', sep='\t')
    #for_lefse_filtered = lib.final_lefse_df(filtered_remove_df, PATH_OUTPUT_DATA, "ALL", lib.SAFE)

Слияние meta_map и meta:
  #SampleID   ID Subgroup       Group         Unnamed: 3
0       10A  10A    MAM-J  OB-BPD/DS   ожирение+резекция
1       10B  10B    MAM-I  OB-BPD/DS   ожирение+резекция
2       10C  10C    MAMLI  OB-BPD/DS   ожирение+резекция
3       10D  10D    MAM-D  OB-BPD/DS   ожирение+резекция
4       10F  10F    Fecal  OB-BPD/DS   ожирение+резекция


In [3]:
safe_df_with_subgroup.head()

,#SampleID,Taxonomy,GROUP,Subgroup,Value,Total,RelativeAbundance
0,10A,Archaea|p_Crenarchaeota|c_Bathyarchaeia|o_uncu...,OB-BPD/DS,MAM-J,0,92085,0.0
1,10A,Archaea|p_Crenarchaeota|c_Bathyarchaeia|o_uncu...,OB-BPD/DS,MAM-J,0,92085,0.0
2,10A,Archaea|p_Crenarchaeota|c_Nitrososphaeria|o_Ni...,OB-BPD/DS,MAM-J,0,92085,0.0
3,10A,Archaea|p_Euryarchaeota|c_Methanobacteria|o_Me...,OB-BPD/DS,MAM-J,0,92085,0.0
4,10A,Archaea|p_Euryarchaeota|c_Methanobacteria|o_Me...,OB-BPD/DS,MAM-J,0,92085,0.0


In [10]:
safe_df_with_subgroup['Subgroup'].unique()

array(['MAM-J', 'MAM-I', 'MAMLI', 'MAM-D', 'Fecal'], dtype=object)

### Subclass


In [2]:
import pandas as pd
import numpy as np
import lib
import json

DATA_PATH = '../data/init/'
NAME = 'only_family_sub_v2'
PATH_OUTPUT_DATA = f'../data/output/{NAME}'
META = 'New_Metadata_BPD'
lib.GROUP = 'Group'

otu_df = lib.read_data(f'{DATA_PATH}OTU.txt', sep='\t')
hiera_blast = lib.read_data(f'{DATA_PATH}hiera_BLAST.txt', sep='\t')
meta_map_init = lib.read_data(f'{DATA_PATH}in.map', sep='\t')
meta = lib.read_data(f'{DATA_PATH}{META}.csv',  sep=',')
genus_df = lib.read_data(f'{DATA_PATH}Genus.txt', sep='\t')
phylum_df = lib.read_data(f'{DATA_PATH}Phylum.txt')
family_df = lib.read_data(f'{DATA_PATH}Family.txt')

with open(f'{DATA_PATH}OTU.biom', 'r') as f:
    BIOM_DATA = json.load(f)

In [3]:
meta_group = meta['Group'].unique().tolist()
pattern = r'^k__\?\|p__\?\|c__\?\|o__\?\|f__\?\|g__\?\|s__$'
FILTERED_GROUPS = meta_group

meta_map_df = lib.preprocess_meta_map(meta_map_init)
meta_map_df = meta_map_df[~meta_map_df['#SampleID'].isin(['29D2', '32D2', '24A2'])]

# Переименовываем столбец в meta, чтобы ключ слияния был 'ID'
meta = meta.rename(columns={'#SampleID': 'ID'})

# Убираем префикс "S" из значений столбца '#SampleID' в meta_map_df, чтобы они совпадали с family_df
meta_map_df['ID'] = meta_map_df['ID'].str.replace('^S', '', regex=True)

merged_meta = pd.merge(meta_map_df, meta, on='ID')
print("Слияние meta_map и meta:")
print(merged_meta.head())

prepare_to_maaslin2, merged_family_df = lib.merge_and_prepare_datasets(meta_map_df, meta, family_df, None)

merged_family_df = lib.update_asv_columns(merged_family_df, BIOM_DATA)

long_format_df = lib.melt_and_prepare_taxonomy(merged_family_df)
taxon_abundance = lib.rel_abundance(long_format_df)
safe_df = taxon_abundance[taxon_abundance['GROUP'].isin(FILTERED_GROUPS)]
safe_df = safe_df[~safe_df['Taxonomy'].str.match(pattern)]


# filtered_df = lib.filter_threshold(taxon_abundance)
# filtered_df_5 = lib.filter_threshold(taxon_abundance, 0.01)
# filtered_remove_df = lib.remove_low_variance_taxa(filtered_df, 0.00001)
# safe_data = filtered_remove_df[filtered_remove_df['GROUP'].isin(FILTERED_GROUPS)]
#to_maaslin_date = lib.to_maaslin_data(family_df, prepare_to_maaslin2)
prepare_to_maaslin2 = prepare_to_maaslin2[prepare_to_maaslin2['GROUP'].isin(FILTERED_GROUPS)]

safe_df_with_subgroup = lib.attach_subgroup_column(
    safe_df=safe_df,
    meta_df=meta,            # ваш DataFrame из CSV с колонкой "Subgroup"
    meta_id='ID',            # как называется ID в meta (по дефолту 'ID')
    df_id='#SampleID',       # как называется ID в safe_df
    subgroup_col='Subgroup'  # какая колонка из meta хотим добавить
)
safe_df_with_subgroup = safe_df_with_subgroup[~safe_df_with_subgroup['Taxonomy'].str.startswith('noHit|p_')]


lib.SAFE=True
if lib.SAFE:
    prepare_to_maaslin2.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/{NAME}.tsv', sep='\t')
    #to_maaslin_date.to_csv(f'{PATH_OUTPUT_DATA}/maaslin/data/maaslin_data.tsv', sep='\t')
    safe_df_with_subgroup.to_csv(f'{PATH_OUTPUT_DATA}/out_data/{NAME}_out.csv', sep='\t')
    #for_lefse_filtered = lib.final_lefse_df(filtered_remove_df, PATH_OUTPUT_DATA, "ALL", lib.SAFE)

Слияние meta_map и meta:
  #SampleID   ID Subgroup       Group         Unnamed: 3
0       10A  10A    MAM-J  OB-BPD/DS   ожирение+резекция
1       10B  10B    MAM-I  OB-BPD/DS   ожирение+резекция
2       10C  10C    MAMLI  OB-BPD/DS   ожирение+резекция
3       10D  10D    MAM-D  OB-BPD/DS   ожирение+резекция
4       10F  10F    Fecal  OB-BPD/DS   ожирение+резекция
